## INSTRUCTIONS

Export your whatsapp chat and save it under "chat.txt".

The function will extract each individual emoji. This means that if you sent "💛💛💛💛💛" on April 3rd 2025, it will be recorded as five individual counts of "💛" on 2025-04-03.

The date format is recorded as following: YYYY-MM-DD.

In [1]:
#!/usr/bin/env python3
# emoji_daily_by_sender.py

import csv
from collections import defaultdict
from datetime import datetime
import re
import emoji

CHAT_FILE = "chat.txt"  # WhatsApp export
OUT_CSV = "emoji_by_day_sender.csv"

# Regex for WhatsApp lines: [14/06/2021 09:56:20] sender: message
LINE_PATTERN = re.compile(r"\[(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}:\d{2})\] (.*?): (.*)")

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%d/%m/%Y")
    except ValueError:
        return None

def extract_emojis(text):
    """Return a list of all emojis in the text, counting repeats individually."""
    return [c for c in text if c in emoji.EMOJI_DATA]

def emoji_unicode(emo):
    """Return a string of Unicode codepoints for the emoji."""
    return '-'.join(f'U+{ord(c):04X}' for c in emo)

def main():
    daily_sender_emojis = defaultdict(lambda: defaultdict(list))

    with open(CHAT_FILE, encoding="utf-8", errors="replace") as f:
        for line in f:
            m = LINE_PATTERN.match(line.strip())
            if m:
                date_str, time_str, sender, message = m.groups()
                date = parse_date(date_str)
                date_key = date.strftime("%Y-%m-%d") if date else "unknown"
                emojis = extract_emojis(message)
                if emojis:
                    daily_sender_emojis[date_key][sender].extend(emojis)

    # Write CSV with one row per emoji occurrence, no count column
    with open(OUT_CSV, "w", newline="", encoding="utf-8") as csvf:
        writer = csv.writer(csvf)
        writer.writerow(["date", "sender", "emoji", "unicode"])
        for date_key in sorted(daily_sender_emojis.keys()):
            for sender in sorted(daily_sender_emojis[date_key].keys()):
                for emo in daily_sender_emojis[date_key][sender]:
                    writer.writerow([date_key, sender, emo, emoji_unicode(emo)])

    print(f"✅ Done! CSV written: {OUT_CSV}")

if __name__ == "__main__":
    main()

✅ Done! CSV written: emoji_by_day_sender.csv
